In [113]:
from common.alpha import Alpha
from common.td import TD

import datetime
import pandas as pd
import numpy as np


from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

pd.set_option("display.max_columns", 100)
init_notebook_mode(connected=True)

In [167]:
alpha = Alpha()

td_client = TD()

In [168]:
end = datetime.datetime.now().timestamp()*1000
start = end - 2592000000
print(start, end)

1560136204053.071 1562728204053.071


In [169]:
# Goal: Pull minute data for the last 5 years for specific ETF stocks
data = td_client.get_price_history(symbol='ACIA', period_type='day', period=10, frequency_type='minute', frequency=1, start_date=int(start), end_date=int(end), needExtendedHoursData=True)

In [170]:
# Converting dict to pandas df
df = pd.DataFrame.from_dict(data)
df.head()

,candles,symbol,empty
0,"{'open': 47.7, 'high': 47.7, 'low': 47.7, 'clo...",ACIA,False
1,"{'open': 47.57, 'high': 47.57, 'low': 47.36, '...",ACIA,False
2,"{'open': 47.57, 'high': 47.68, 'low': 47.57, '...",ACIA,False
3,"{'open': 47.685, 'high': 47.755, 'low': 47.53,...",ACIA,False
4,"{'open': 47.94, 'high': 47.99, 'low': 47.765, ...",ACIA,False


In [171]:
# Expanding 'candles' col to multiple columns
df = pd.concat([df.drop(['candles'], axis=1), df['candles'].apply(pd.Series)], axis=1)
df.head()

,symbol,empty,open,high,low,close,volume,datetime
0,ACIA,False,47.700,47.700,47.700,47.700,125.0,1.560172e+12
1,ACIA,False,47.570,47.570,47.360,47.360,4071.0,1.560173e+12
2,ACIA,False,47.570,47.680,47.570,47.680,801.0,1.560173e+12
3,ACIA,False,47.685,47.755,47.530,47.755,400.0,1.560174e+12
4,ACIA,False,47.940,47.990,47.765,47.765,988.0,1.560174e+12


In [172]:
# Create 'date' col from 'datetime'. Converting from epoch time from milliseconds to seconds, then to date
df['date'] = df['datetime'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M'))
df = df.set_index('date')
df.head()

,symbol,empty,open,high,low,close,volume,datetime
date,,,,,,,,
2019-06-10 08:14,ACIA,False,47.700,47.700,47.700,47.700,125.0,1.560172e+12
2019-06-10 08:30,ACIA,False,47.570,47.570,47.360,47.360,4071.0,1.560173e+12
2019-06-10 08:31,ACIA,False,47.570,47.680,47.570,47.680,801.0,1.560173e+12
2019-06-10 08:32,ACIA,False,47.685,47.755,47.530,47.755,400.0,1.560174e+12
2019-06-10 08:35,ACIA,False,47.940,47.990,47.765,47.765,988.0,1.560174e+12


In [173]:
INCREASING_COLOR = 'g'
DECREASING_COLOR = 'r'

In [174]:
start_chart = datetime.datetime.fromtimestamp((end - 86400000)/1000.0).strftime('%Y-%m-%d %H:%M')
df_trim = df[df.index >= start_chart]

data = [ dict(
    type = 'candlestick',
    open = df_trim['open'],
    high = df_trim['high'],
    low = df_trim['low'],
    close = df_trim['close'],
    x = df_trim.index,
    yaxis = 'y2',
    name = 'Historic Data',
    increasing = dict( line = dict( color = INCREASING_COLOR ) ),
    decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
) ]

layout=dict()

fig = dict( data=data, layout=layout )

In [175]:
fig['layout'] = dict()
fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ), autorange = True )
fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = False, autorange = True )
fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

In [176]:
rangeselector=dict(
    visibe = True,
    x = 0, y = 0.9,
    bgcolor = 'rgba(150, 200, 250, 0.4)',
    font = dict( size = 13 ),
    buttons=list([
        dict(count=1,
             label='reset',
             step='all'),
        dict(count=1,
             label='1 yr',
             step='year',
             stepmode='backward'),
        dict(count=3,
            label='1 month',
            step='month',
            stepmode='backward'),
        dict(count=1,
            label='1 day',
            step='day',
            stepmode='backward'),
        dict(step='all')
    ]))
    
fig['layout']['xaxis']['rangeselector'] = rangeselector

In [177]:
df['9_day_ema'] = df['close'].ewm(span=9, adjust=False).mean()
df['20_day_ema'] = df['close'].ewm(span=20, adjust=False).mean()

df_trim = pd.merge(df_trim, df[['9_day_ema', '20_day_ema']], how='left', left_index=True, right_index=True)

In [179]:
# Add EMA lines
df_trim['9_day_ema2'] = df_trim['close'].ewm(span=9, adjust=False).mean()
df_trim['20_day_ema2'] = df_trim['close'].ewm(span=20, adjust=False).mean()

mv_y = df_trim['9_day_ema']
mv_x = list(df_trim.index)

fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#2ca02c' ),
                         fill = None,
                         yaxis = 'y2', name='9 Day Exp Moving Average' ) )

mv_y = df_trim['20_day_ema']
mv_x = list(df_trim.index)

fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#17becf' ),
                         fill = None,
                         yaxis = 'y2', name='20 Day Exp Moving Average' ) )

In [161]:
df_trim['cross'] = np.where(df_trim['9_day_ema'] - df_trim['20_day_ema'] >= df_trim['close']*.0005, df_trim['close']*1.05, None)

mv_y = df_trim['cross']
mv_x = list(df_trim.index)

fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 3 ),
                         marker = dict( color = '#00FF00' ),
                         yaxis = 'y2', name='Buy' ) )

/home/rahul/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [162]:
fig['data'].append( dict( x=df_trim.index, y=df_trim['volume'],                         
                         marker=dict( color= '#6B5B95' ),
                         type='bar', yaxis='y', name='Volume' ) )

In [163]:
iplot(fig, filename='charts/dgaz_test2', validate=False)